In [14]:
import cv2
import numpy as np
import mediapipe as mp
import pyautogui
import math
import time
from IPython.display import clear_output
from pynput.keyboard import Controller, Key

COMMON CODE

In [15]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [16]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

In [17]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return lh

In [18]:
def extract_thumb_keypoints(results):
    keypoints = []
    for data_point in results.left_hand_landmarks.landmark:
        keypoints.extend([data_point.x, data_point.y, data_point.z])
    return keypoints

KEYBOARD CODE

In [19]:
class Button():
    def __init__(self, txt, pos, size = 38):
        self.size = size
        self.pos = pos
        self.text = txt

In [20]:
def place_keys(img, keys, i, j):   
    for key in keys:
        (img, key, [i, j])
        i += 100
    return img

In [21]:
def click_btn(image, btn, text, key):
    mask = image.copy()
    cv2.circle(mask, btn.pos, btn.size, (255,255,255), thickness=-1)
    image = cv2.addWeighted(mask, 1, image, 1 - 1, 0)
    if btn.text == "<-":
        if text != "":
            text = text[:-1]
            key.press(Key.backspace)
    elif btn.text == "_":
        text += " " 
        key.press(Key.space)
    else:
        text += btn.text
        key.press(btn.text)
    return image, text

In [22]:

def activate_btn(image, btn, il, ml, text, key):
    if abs(il[0]-btn.pos[0]) <= btn.size and abs(il[1]-btn.pos[1]) <= btn.size:
        mask = image.copy()
        cv2.circle(mask, btn.pos, btn.size, (255,255,255), thickness=-1)
        image = cv2.addWeighted(mask, 0.5, image, 1 - 0.5, 0)
        d = math.dist(il, ml)
        if d < 50:
            
            image, text = click_btn(image, btn, il, text, key)
            time.sleep(0.2)
    return image, text
                    

MOUSE


In [23]:
def right_move_mouse(landmarks, image, frame):
    clear_output(wait=True)
    # frameH, frameW, _ = frame.shape
    fingers = []

    for i in range(4, 21, 4): fingers.append([int(landmarks.landmark[i].x * frame[0] ), int(landmarks.landmark[i].y * frame[1])])

    # thumb = fingers[0]
    # index = fingers[1]
    # middle = fingers[2]
    # ring = fingers[3]
    # pinky = fingers[4]

        
    cv2.circle(image, fingers[0], 10, (0, 255, 0), thickness = 2)
    cv2.circle(image, fingers[1], 10, (0, 34, 234), thickness = 2)

    move_dist = math.dist( fingers[0] , fingers[1])
    left_click_dist = math.dist(fingers[0], fingers[2])
    ring_click_dist = math.dist(fingers[0], fingers[3])
    drag_dist = math.dist(fingers[0], fingers[4])

    mouse_x, mouse_y = pyautogui.position()

    if ring_click_dist < 30:
        pyautogui.rightClick(mouse_x, mouse_y)
    if left_click_dist < 30:
        pyautogui.click()

    if move_dist < 80:
        return 'move'
    if drag_dist < 80:
        return 'drag'

In [24]:
def left_move_mouse(landmarks, image, frame, is_clicked, pil, initial_time):
    clear_output(wait=True)
    time_spent = (time.time()-initial_time)*100
    mouse_x, mouse_y = pyautogui.position()
    # frameH, frameW, _ = frame.shape
    
    il = [int(landmarks.landmark[8].x * frame[0]), int(landmarks.landmark[8].y * frame[1])]

    dist_x = il[0] - pil[0]
    dist_y = il[1] - pil[1]
    dist = math.dist(pil, il)
    speed = int((dist/time_spent))

    if(not speed): speed = 1 
    cv2.circle(image, (il[0], il[1]), 10, (0, 34, 234), thickness = 2)
    if is_clicked == 'move' or is_clicked == 'drag':
        x = mouse_x + (dist_x*speed)
        y = mouse_y + (dist_y*speed)
        if x < 2: x = 2
        if y < 2: y = 2
        if x > 1920: x = 1920
        if y > 1075: y = 1075
        if is_clicked == 'move':
            pyautogui.moveTo(x, y)
        if is_clicked == 'drag':
            pyautogui.dragTo(x, y, 0.2, button='left')        
    
    return il, time.time()

In [25]:
#keyboard
keys = [['Q', 'W', 'E', 'R', 'T', 'Y', 'U'],
        ['I', 'O', 'P', 'A', 'S', 'D', 'F'],
        ['G', 'H', 'J', 'K', 'L', 'Z', 'X'],
        ['C', 'V', 'B', 'N', 'M', '_', '<-']]

tense = "|"
keyboard = Controller()
btnList = []
p_time = time.time()

CA = Button('CA', (940, 90))

y = 50

for i in range(4):
    x = 200
    for key in keys[i]:
        btnList.append(Button(key, (x, y)))
        x += 100
    y += 100

#mouse
    
trX, trY = 0, 0
    
il = [0, 0]

initial_time = time.time()
switch_time = time.time()
flick_time = time.time()

count = 0

switch = True
cap = cv2.VideoCapture(0)


try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        while cap.isOpened():
            ret, image = cap.read()
            clear_output(wait=True)
            # image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if switch:
                frameW, frameH = 1080, 784
                image = cv2.resize(image,(1080, 784))
            
            image, results = mediapipe_detection(image, holistic)
            left_landmarks = results.right_hand_landmarks
            right_landmarks = results.left_hand_landmarks

            if switch:
                
                # for flickering
                if (time.time() - flick_time) > 0.3:
                    flick_time = time.time()
                    if tense[len(tense)-1:] == "|":
                        tense = tense[:-1]
                    else: 
                        tense += "|"
                    
                
                # for placing buttons
                for btn in btnList:
                    mask = image.copy()
                    cv2.circle(mask, btn.pos, btn.size, (255,255,255), thickness=-1)
                    image = cv2.addWeighted(mask, 0.7, image, 1 - 0.7, 0)
                    cv2.putText(image, btn.text, (btn.pos[0]-10, btn.pos[1]+15), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,0), 2)

                if left_landmarks:
                    tlX = int(left_landmarks.landmark[4].x*frameW)
                    tlY = int(left_landmarks.landmark[4].y*frameH)
                    ilX = int(left_landmarks.landmark[8].x*frameW)
                    ilY = int(left_landmarks.landmark[8].y*frameH)

                    

                    cv2.circle(image, (ilX, ilY), 15, (0, 255, 0), thickness=5)

                    # for activating button when hand is close to it
                    for btn in btnList:
                        if abs(ilX-btn.pos[0]) <= btn.size and abs(ilY-btn.pos[1]) <= btn.size:
                            mask = image.copy()
                            cv2.circle(mask, btn.pos, btn.size, (0,0,0), thickness=-1)
                            image = cv2.addWeighted(mask, 0.8, image, 1 - 0.8, 0)
                            cv2.putText(image, btn.text, (btn.pos[0]-10, btn.pos[1]+15), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
                        

                    if right_landmarks:
                        irX = int(right_landmarks.landmark[8].x*frameW)
                        irY = int(right_landmarks.landmark[8].y*frameH)
                        trX = int(right_landmarks.landmark[4].x*frameW)
                        trY = int(right_landmarks.landmark[4].y*frameH)
                        
                        cv2.circle(image, (trX, trY), 15, (0, 255, 0), thickness=5)
                        cv2.circle(image, (irX, irY), 15, (0, 255, 0), thickness=5)

                        for btn in btnList:
                            d = int(math.dist([irX, irY], [trX, trY]))
                            if abs(ilX-btn.pos[0]) <= btn.size and abs(ilY-btn.pos[1]) <= btn.size:
            
                                if d < 50 and (time.time()-p_time) > 0.8:
                                    if tense[len(tense)-1:] == "|": tense = tense[:-1]
                                    image, text = click_btn(image, btn, tense, keyboard)
                                    count += 1
                                    tense = text
                                    p_time = time.time()

                            if abs(ilX-90) <= 40 and abs(ilY-75) <= 25 and d < 50:
                                while tense:
                                    keyboard.press(Key.backspace)
                                    tense = tense[:-1]
                                
                                tense = "|"
                                
                        d = int(math.dist([tlX, tlY], [trX, trY]))

                        # print(time.time()-switch_time)
                        # for switching between mouse and keyboard
                        if d < 40 and (time.time() - switch_time) > 2:
                            switch_time = time.time()
                            switch = False
                            
                                
                # text box
                # x is left right, y is up down
                cv2.rectangle(image, (20, 450), (700, 550), (0, 0, 0), cv2.FILLED)
                cv2.putText(image, tense, (50, 530), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)

                # Right hand disclaimer box
                cv2.rectangle(image, (700, 400), (1080, 780), (0, 0, 0), 3)
                cv2.putText(image, 'Put your right', (730, 450), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                cv2.putText(image, 'hand here', (750, 490), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                
                # clear button
                cv2.rectangle(image, (50, 50), (130, 100), (0, 0, 0), cv2.FILLED)
                cv2.putText(image, 'CA', (70, 90), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)

            else:
                frameH, frameW, _ = image.shape
                frame = [frameW, frameH]
                
                is_clicked = False

                if right_landmarks: 
                    is_clicked =  right_move_mouse(right_landmarks, image, frame)
                    trX = int(right_landmarks.landmark[4].x*frameW)
                    trY = int(right_landmarks.landmark[4].y*frameH)

                if left_landmarks: 
                    il, initial_time = left_move_mouse(left_landmarks, image, frame, is_clicked, il, initial_time)
                    tlX = int(left_landmarks.landmark[4].x*frameW)
                    tlY = int(left_landmarks.landmark[4].y*frameH)
                    ilX = int(left_landmarks.landmark[8].x*frameW)
                    ilY = int(left_landmarks.landmark[8].y*frameH)

                    d = int(math.dist([tlX, tlY], [trX, trY]))

                    if d < 40 and (time.time() - switch_time) > 2:
                        switch_time = time.time()
                        switch = True
                    
            cv2.imshow("Hand Tracking", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
finally:
    cap.release()
    cv2.destroyAllWindows()

In [26]:
x = "abc|"

print(x[:-1])

abc
